# KVLCC2

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
import matplotlib
#plt.style.use('presentation')

from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig


from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.models.regression import MotionRegression, Regression

from src.parameters import df_parameters
from src.substitute_dynamic_symbols import run
from src.models.diff_eq_to_matrix import DiffEqToMatrix
p = df_parameters["symbol"]
import statsmodels.api as sm

from src.models.force_from_motion import predict_force

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)
from src.prime_system import PrimeSystem
from wPCC_pipeline.pipelines.kvlcc2.nodes import calculate_thrust

from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import initial_state, extended_kalman_filter, extended_kalman_smoother, guess_covariance_matrixes


| Column | Unit | Meaning | Reference |
| ------ | ---- | ------- | ----------|
| 1 | s | time | manoeuvre starts at zero i.e. the rudder starts to move at t=0. Time step is about 0.135(full scale) |
| 2 | m | x-position | position of ship origo  in Earth-fixed coordinate system(North). |
| 3 | m | y-position | position of ship origo in Earth-fixed coordinate system(East) |
| 4 | deg | heel angle | starboard side in the water is a positive |
| 5 | deg | heading angle | bow to starboard is positive |
| 6 | m/s | long. velocity | speed through the water of ship origo |
| 7 | m/s | transv. velocity | speed through the water of ship origo |
| 8 | deg/s | roll velocity | ship origo, starboard downwards is positive |
| 9 | deg/s | yaw velocity | ship origo, bow towards starboard is positive |
| 10 | deg | rudder angle | trailing edge to portside is positive |
| 11 | RPM | prop. revolutions | positive clockwise seen from aft. |


In [ ]:
columns = [
'time',
'x0',
'y0',
'phi',
'psi',
'u',
'v',
'p',
'r',
'delta',
'rev',
]

df = pd.read_csv("../data/01_raw/kvlcc2/MARIN_FREE_KVLCC2_zz_-20_m.dat", names=columns, header=1, sep='\t')
df.dropna(how='all', inplace=True)

mask = df['time'].diff()!=0
df = df.loc[mask]
df.set_index('time', inplace=True)
df.sort_index(inplace=True)
df.index = pd.to_timedelta(df.index, unit='s')
df = df.resample('0.1S').interpolate().resample('5S').mean()
df.index = df.index.total_seconds()

angles = ['phi','psi','r','delta']
df[angles] = np.deg2rad(df[angles])

In [ ]:
df.isnull().any()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,15)
ax.axis('equal')
df.plot(x='y0', y='x0', ax=ax)


In [ ]:
df.plot(y='y0', style='.-')

In [ ]:
df.iloc[-100:]

In [ ]:
%reload_kedro
ship_data = catalog.load("kvlcc2.ship_data")

In [ ]:
np.sqrt(ship_data['I_z']/ship_data['m'])/ship_data['L']

In [ ]:
ship_data_wpcc = catalog.load("wpcc.ship_data")
np.sqrt(ship_data_wpcc['I_z']/ship_data_wpcc['m'])/ship_data_wpcc['L']

In [ ]:
ps = PrimeSystem(**ship_data)

In [ ]:
shipdata_prime = ps.prime(ship_data)

In [ ]:
ship_data['m']/(1/2*1000*(ship_data['L']**2*ship_data['T']))

In [ ]:
0.235/(1/2*(2.909**3))

In [ ]:
I_z = ps._unprime(0.011,unit = "inertia_moment")
np.sqrt(I_z/ship_data['m'])/ship_data['L']

In [ ]:
k_z = 0.25
ship_data['m']*(ship_data['L']*k_z)**2

In [ ]:
ship_data['m']*(ship_data['L']*0.5)**2

In [ ]:
prime_1_to_2 = ship_data['T']/ship_data['L']
Xudot_prime2 = 0.022
Yvdot_prime2 = 0.223
Nrdot_prime2 = 0.011

Xudot_prime = Xudot_prime2*prime_1_to_2
Yvdot_prime = Yvdot_prime2*prime_1_to_2
Nrdot_prime = Nrdot_prime2*prime_1_to_2


In [ ]:
Xudot_prime

In [ ]:
Yvdot_prime

In [ ]:
Nrdot_prime

In [ ]:
x_p = 5.2/ship_data['scale_factor'] - ship_data['L']/2
x_p

In [ ]:
x_r = - ship_data['L']/2
x_r

In [ ]:
%reload_kedro

id = "MARIN_FREE_KVLCC2_tc_35_m"
df = catalog.load(f"kvlcc2.{id}.raw_data_unformated")
df_raw = catalog.load(f"kvlcc2.{id}.data")


df.dropna(how="all", inplace=True)

df.plot(x='time', y='u')

mask = df["time"].diff() != 0
df = df.loc[mask].copy()

df.set_index("time", inplace=True)
df.sort_index(inplace=True)
df.index = pd.to_timedelta(df.index, unit="s")
df = df.resample("0.1S").interpolate().resample("0.5S").mean()

scale_factor = ship_data["scale_factor"]
df.index = df.index.total_seconds() / np.sqrt(scale_factor)

df['r'] = np.deg2rad(df['r'])
df["r"]*= np.sqrt(scale_factor)
df["u"]/= np.sqrt(scale_factor)
df["v"]/= np.sqrt(scale_factor)


df.head()


In [ ]:
fig,ax=plt.subplots()
df.plot(y='r', ax=ax)
df_raw.plot(y='r', style='--', ax=ax)

fig,ax=plt.subplots()
df.plot(y='u', ax=ax)
df_raw.plot(y='dx0', style='--', ax=ax)

fig,ax=plt.subplots()
df.plot(y='v', ax=ax)
df_raw.plot(y='dy0', style='--', ax=ax)


In [ ]:
%reload_kedro

ek = catalog.load("kvlcc2.vmm_linear.ek")
ek_covariance_input = catalog.load("params:kvlcc2.initial.ek_covariance_input")
hydrodynamic_derivatives = catalog.load("kvlcc2.initial_parameters")

In [ ]:
ek_covariance_input = {
    'process_variance': 
    {
        'u': 0.00001, 
        'v': 0.00001, 
        'r': 0.00001,
    },
    'measurement_error_max': 
    {
        'positions': 0.1, 
        'psi': 0.1
    }
}


In [ ]:
error_max_psi = np.deg2rad(ek_covariance_input['measurement_error_max']["psi"])
sigma_psi = error_max_psi / 3
variance_psi = sigma_psi ** 2
variance_psi

In [ ]:
covariance_matrixes = guess_covariance_matrixes(ek_covariance_input=ek_covariance_input, data = df_raw)

In [ ]:
covariance_matrixes['P_prd']

In [ ]:
covariance_matrixes['Qd']

In [ ]:
covariance_matrixes['Rd']

In [ ]:
x0 = initial_state(data=df_raw)
x0

In [ ]:
ek_filtered, data_ek_filter, time_steps = extended_kalman_filter(ek=ek, data=df_raw, covariance_matrixes=covariance_matrixes, x0=x0, hydrodynamic_derivatives=hydrodynamic_derivatives)

In [ ]:
dataframes = {
    'raw':df_raw,
    'ek1':data_ek_filter,
}
plot(dataframes, fig_size=(15,10), keys=['u','v','r','psi'], time_window=[0,1000]);

In [ ]:
df.plot(y='psi')

In [ ]:
%reload_kedro

#id = "MARIN_FREE_KVLCC2_tc_35_m"
id = "MARIN_FREE_KVLCC2_zz_20_m"
dataframes = {}
dataframes[id] = catalog.load(f"kvlcc2.{id}.raw_data")

id = "MARIN_FREE_KVLCC2_zz_-20_m"
#df_ = catalog.load(f"kvlcc2.{id}.data_ek_smooth")
df_ = catalog.load(f"kvlcc2.{id}.raw_data")
df_['y0']*=-1
df_['psi']*=-1
df_['delta']*=-1
df_['r']*=-1
df_['v']*=-1
#df_['r1d']*=-1
#df_['v1d']*=-1

dataframes[id] = df_

track_plots(dataframes=dataframes, lpp=ship_data['L'], beam=ship_data['B']);
plot(dataframes);

In [ ]:
%reload_kedro

id = "MARIN_FREE_KVLCC2_tc_35_m"
#id = "MARIN_FREE_KVLCC2_zz_20_m"

df_raw = catalog.load(f"kvlcc2.{id}.raw_data")
data = catalog.load(f"kvlcc2.{id}.data")
df_smooth = catalog.load(f"kvlcc2.initial.{id}.data_ek_smooth")
df_ek = catalog.load(f"kvlcc2.initial.{id}.data_ek_filter")

df_smooth2 = catalog.load(f"kvlcc2.updated.{id}.data_ek_smooth")
df_ek2 = catalog.load(f"kvlcc2.updated.{id}.data_ek_filter")

#df_raw.index = df_raw.index-4
dataframes = {
              #'lowpass':data,
              'df_ek': df_ek,
              'smooth' : df_smooth,
              'df_ek2': df_ek2,
              'smooth2' : df_smooth2,
    
              'raw':data,
              
             }

styles = {

    'raw':{'alpha':0.5},
}

In [ ]:
track_plots(dataframes=dataframes, lpp=ship_data['L'], beam=ship_data['B']);

In [ ]:
plot(dataframes=dataframes, keys=["thrust", "psi", "u", "v", "r", "u1d","v1d","r1d"], styles=styles);

In [ ]:
from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import extended_kalman_filter, extended_kalman_smoother

In [ ]:
%reload_kedro
ek = catalog.load("kvlcc2.vmm_martin.ek")
covariance_matrixes = catalog.load("kvlcc2.vmm_martin.covariance_matrixes")
x0 = catalog.load(f"kvlcc2.{id}.x0")
parameters = catalog.load(f"kvlcc2.vmm_martin.joined.derivatives")

In [ ]:
ek.parameters.update(parameters['regressed'])

In [ ]:
ek_filtered, data_ek_filter, time_steps = extended_kalman_filter(ek=ek, data=data, covariance_matrixes=covariance_matrixes, x0=x0)

In [ ]:
ek_smooth, data_ek_smooth = extended_kalman_smoother(ek=ek, data=data, time_steps=time_steps, covariance_matrixes=covariance_matrixes)

In [ ]:
dataframes['smooth2'] = data_ek_smooth

In [ ]:
dataframes = {'raw':df_raw,
              'df_ek': df_ek,
              #'smooth' : df_smooth,
              'df_ek2': data_ek_filter,
              #'smooth2' : data_ek_smooth,
                            
             }

styles = {

    'df_ek':{'alpha':0.5},
}

plot(dataframes=dataframes, keys=["thrust", "psi", "u", "v", "r", "delta"], styles=styles);

In [ ]:
dataframes = {'raw':df_raw,
              #'df_ek': df_ek,
              'smooth' : df_smooth,
              #'df_ek2': data_ek_filter,
              'smooth2' : data_ek_smooth,
                            
             }

styles = {

    'df_ek':{'alpha':0.5},
}

plot(dataframes=dataframes, keys=["thrust", "psi", "u", "v", "r", "delta"], styles=styles);

In [ ]:
x = [1,2,3,4]
y = [0,1,0,0]
dydx = np.gradient(y, x)

fig,ax=plt.subplots()
ax.plot(x,y,'s-')
ax2 = ax.twinx()
ax2.plot(x,dydx,'.-')